In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("Project1.ipynb")

In [ ]:
rng_seed = 42

# Project 1

# <em> Optimization, Monte Carlo Problems, pocoMC </em>
<br>

Welcome to your first project. Unlike the homework, you will generally receive less guidance on projects but are still expected to do the work required. This project primarily covers sampling and the use of MCMC methods. You will be analyzing some data using a number of methods for parameter inference.

Though there will be physics involved in these projects, the physical quantities and equations are generally given to you, while you are expected to use the physics equations to produce data analysis results. Accordingly, autograder will be less through for these questions, and most of the project will be graded manually, so make sure you include documentation with your code!


### Imports

In [ ]:
import numpy as np
from scipy.integrate import quad
#For plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

***

# Problem 1: Constraints on Galaxy Clustering

Parameter constraints methods are used in many physical problems. In this project, we consider the use of two different methods, Maximum Likelihood Estimation (MLE) and Markov Chain Monte Carlo (MCMC), to constrain Halo Occupation Distribution (HOD) models.

Here, we consider constraints on the Halo Occupation Distribution (HOD) models for projected galaxy clustering.

HOD models are used to interpret the relation of galaxy distributions in large scale structure to the dark matter distribution. They describe related properties of the galaxy distribution within a dark matter halo: The probability distribution of the number of galaxies within that halo in relation to the mass of the halo, the distribution in space of galaxies withing the DM halo and the distribution in velocity of galaxies in the halo.

Traditional HOD models make two assumptions: (1) that all galaxies reside in dark matter haloes and are biased tracers of the underlying matter density and (2) galaxies occupy halos **only** as a function of the halo mass $M_{vir}$. However, these are not necessarily correct, an effect called *halo assembly bias* could be significant. They also add additional parameters to the HOD model. In this problem, we will infer some HOD model parameters given some input data using MCMC.

First, we will describe the parameters. HOD models usually treat central galaxies and satellite galaxies in a halo separately, and the occupation statistics of the central galaxy will be different than the occupation statistics of the satellite galaxies in the halo.
The average number of central galaxies in a halo of mass $M$ is given by

$$\langle N_{cen}\rangle_{M} = \frac12 \left[1 + \mathrm{erf}\left( \frac{\log M - \log M_{\mathrm{min}}}{\sigma_{\log M}}\right)\right]$$

While the average number of satellite galaxies in a halo of mass $M$ is given by

$$\langle N_{sat}\rangle_{M} = \left[\frac{M-M_0}{M_1}\right]^\alpha$$

The parameter $M_{\mathrm{min}}$ describes the halo mass at which the halo has 50\% probability to host a central galaxy, while the parameter $\sigma_{\log M}$ can be considered the rate at which the likelihood of the central galaxy to exist increases alongside halo mass. The parameter $M_1$ is the halo mass at which an average halo would host one satellite galaxy, while $M_0$ is the halo mass at which an average halo would not have a satellite galaxy. $\alpha$ is the power-law exponent relation parameter between the number of satellites and the halo mass. Lastly, we also have the two parameters $A_{cen}$ and $A_{sat}$, quantify describe the halo assembly bias.

In summary, we take a likelihood model of the Halo Occupation Distribution and infer the following parameters:
$$[\log(M_{\mathrm{min}}), \sigma_{\log M}, \log(M_0), \log(M_1), \alpha, A_{cen}, A_{sat}]$$

(This problem is based on the following paper: https://arxiv.org/pdf/1606.07817.pdf)

In [ ]:
import numpy as np
import corner
from scipy.stats import uniform
from tabcorr import TabCorr
from halotools.empirical_models import HodModelFactory
from halotools.empirical_models import AssembiasZheng07Cens
from halotools.empirical_models import AssembiasZheng07Sats

import numdifftools

In [ ]:
# Load data
halotab = TabCorr.read('./zentner19data/bolplanck.hdf5')
cens_occ_model = AssembiasZheng07Cens()
sats_occ_model = AssembiasZheng07Sats()
model = HodModelFactory(centrals_occupation=cens_occ_model,
                        satellites_occupation=sats_occ_model)

n_obs = 6.37e-3
n_err = 0.75e-3
wp_obs = np.genfromtxt('./zentner19data/wp_dr72_bright0_mr20.0_z0.106_nj400')[:, 1] # Observed galaxy two-point function
wp_cov = np.genfromtxt('./zentner19data/wpcov_dr72_bright0_mr20.0_z0.106_nj400') # Observed galaxy two-point function covariance.

First, we would like to define priors on the model parameters. Since we don't know too much beyond the range of the priors, we will define uniform priors for each parameter as given:

\begin{array}{ c|c }
 \text{Parameter} & \text{Prior Interval} \\
 \hline
 \log(M_{\mathrm{min}}) & [9.0, 14.0] \\
 \sigma_{\log M} & [0.01, 1.5]\\
 \log(M_0) & [9.0, 14.0]\\
 \log(M_1) & [10.7, 15.0]\\
 \alpha & [0.0, 2.0]\\
 A_{cen} & [-1.0, 1.0] \\
 A_{sat} & [-1.0, 1.0]
\end{array}

Furthermore, to constrain parameters, we need a description of the likelihood function of parameters. Here, following the accompanying paper to this problem, we define the likelihood of the form $\mathit{L} \propto e^{-\chi^2/2}$, with the $\chi^2$ given as

$$\chi^2 = \Delta w_{p,i}\, [C^{-1}]_{ij}\, \Delta w_{p, j} + \frac{(n_g^{\text{mock}} - n_g^{\text{obs}})^2}{\sigma^2_n}$$

with $ \Delta w_{p,i} \equiv w_p^{\text{mock}}(r_{p, i}) -  w_p^{\text{obs}}(r_{p, i})$ being the difference between the projected and observed two point correlation functions $w_{p}(r_{p,i})$. The observed $w_p^{\text{obs}}$ is given to you while the predicted $w_p^{\text{mock}}$ is received from the model (see below). The $r_{p,i}$ represents the 12 $r$ bins, and you can see that the given $w_p$ are indeed 12 dimensional. $C$ is the covariance matrix of the measurements (also given), and the term $\frac{(n_g^{\text{mock}} - n_g^{\text{obs}})^2}{\sigma^2_n}$ is the contribution from the difference between the predicted and measured galaxy number densities. ($n_g^{\text{mock}}$ is given by the model, while $\sigma_n$ and $n_g^{\text{obs}}$ are given from the data).

## **Problem Goals**

Using the likelihood function and priors given,  you are expected to compute constraints on the halo parameters in two different methods: first, using a **Maximum Likelihood Estimate (MLE)** approach where you assume the data is Gaussian, then using the **Metropolis-Hastings algorithm** for **Markov Chain Monte Carlo (MCMC)**. You are then expected to compare the results from these two methods and briefly discuss the differences. Lastly, compare the results to Figure 6 from https://arxiv.org/pdf/1606.07817.pdf.

<!-- BEGIN QUESTION -->

### Part 1. Define the Log Likelihood function


Given a set of data $X$ and a model $M(\theta)$ on this data with parameters $\theta$, we can define the likelihood function as the probability that the model with parameters $\theta$ describes the data:
$$L(\theta; X) = P(X | \theta)$$

Usually, we work in the log space as the log-likelihood is easier to define and optimize. As the log function is monotonic, anything that yields the maximum or minimum log-likelihood would then also yield the maximum or minimum likelihood. The log-likelihood is defined as
$$\ell(\theta;X) = \log(L(\theta; X))$$


<span style="color:blue"> <i> Using this chi square given above, define the priors and the log likelihood function.



In [ ]:
# Define log-likelihood function
names = ['logMmin', 'sigma_logM', 'logM0', 'logM1', 'alpha',
         'mean_occupation_centrals_assembias_param1',
         'mean_occupation_satellites_assembias_param1']
bounds = ...

def log_likelihood(theta):

    theta = theta.copy() # copy the x so as to not change the original
    model.param_dict.update(dict(zip(names, theta))) # update model with input parameters
    n_mod, wp_mod = halotab.predict(model) # get predicted results
    
    return ...


In [ ]:
grader.check("q1.1")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Part 2. Maximum Likelihood Estimate / Maximum A Posteriori

Given a log likelihood function $\ell(\theta; X)$, in the frequentist set-up we can calculate the maximum likelihood estimate (MLE) of our parameters $\theta$ by finding the MLE best fit parameters $\widehat{\theta}$

$$\widehat{\theta} = \mathrm{argmax}_\theta \; \ell(\theta; X)$$

However, in this problem, we don't just want to find the *best fit parameters* $\widehat{\theta}$, but we also want to find the *uncertainties* of these best fit parameters, as well as the covariances of these parameters and their distribution in parameter space.

The Fisher Information Matrix could help us find the parameter covariance. Recall that the Fisher Matrix is defined as
$$F_{ij} = - \left\langle \frac{∂^2}{∂\theta_i∂\theta_j} \ell(\theta; X)\right\rangle_{\theta_{MLE}}$$
(Note that as described in the lecture notes, the fisher matrix is evaluated at the best-fit MLE parameters) The Fisher matrix also has the property such that its inverse yields the covariance matrix.
$$C^{-1} = F$$

To compute the Fisher information, we make use of the Hessian matrix of the log-likelihood. This is defined as
$$H_{ij} = \frac{∂^2}{∂\theta_i∂\theta_j} \ell(\theta; X)$$ which means that the Fisher Information matrix is the negative expectation of the Hessian
$$F = -\left\langle H \right\rangle$$

Note: this is only true in the assumption of large, independent samples from the likelihood, when distribution on $\theta$ exhibits asymptotic normality.
$$\hat{\theta} \sim N\left(\theta, \frac{F(\theta)^{-1}}{n} \right)$$
By the Laplace approximation, this indicates that the Hessian would converge to the Fisher Information Matrix.

In practice, this symbolizes that the Hessian of the log likelihood can be used to approximate the Fisher matrix. Under the assumption that the parameter space is Gaussian distributed then, we can use the Hessian of the likelihood to calculate the covariance matrix and plot Gaussian stair plots showing the distribution in parameter space.

**The goal of this section is to accomplish this task. Find the parameters that maximize the log likelihood within the prior bounds given above, and plot the parameter distributions for the parameters given, using the prior bounds as bounds of each of the stair plots.** To help in the plotting part, a skeleton stair plot code is provided, but you will have to modify the code to fit the parameters. (Note that technically, we are conducting Maximum A Posteriori (MAP) estimation of parameters as we're incorporating the uniform priors given earlier)

Hints:

1.   `scipy.optimize` allows the use of optimization methods with bounds on optimization parameters. Read the documentation and use a method that is able to do bounded optimization on the likelihood.
2.   To compute the Hessian on the log-likelihood function, use numerical finite difference methods and not automatic differentiation methods. This is because the likelihood function you constructed in Part 1 utilises packages which are not automatically differentiable by autograd or pytorch. The package `numdifftools` contains the function `numdifftools.Hessian` which will help you in this area.

In [ ]:
# Use these labels for the parameters. Make sure you use the parameters in this order for the autograder!

labels = [r'$\log M_{min}$', r'$\sigma_{\log M}$',r'$\log M_0$',r'$\log M_1$',r'$\alpha$',r'$A_c$',r'$A_s$'] 

opt_p = ...

print(opt_p)

for i in range(len(labels)):
    print(r"MLE value of %s = %.5f" %(..., ...))

In [ ]:
...

In [ ]:
# Triangle Plot (Original skeleton code by Nicholas Kern)
# Only a suggestion. You can create your own if you wish.

fig, axes = plt.subplots(6,6,figsize=(12,12))
fig.subplots_adjust(wspace=0, hspace=0)

p_tex = np.array([r'$H_0$', r'$\Omega_bh^2$',r'$\Omega_ch^2$',r'$n_s$',r'$10^9 A_s$',r'$\tau$'])

for i in range(6):
    for j in range(6):
        ax = axes[i, j]
        if j > i:
            ax.axis('off')
            continue
        elif i == j:
            # diagonal part
            ax.grid(True)
            xarr = ...
            yarr = ...
            ax.plot(...)
            ax.set_xlim(...)
            ax.set_xticks(...)
            ax.set_yticklabels([])
            ax.set_xticklabels([])
        else:
            # off-diagonal part
            ax.grid(True)

            # Covariance matrix
            CovM = ...

            # Get eigenvalue/vector using svd
            eigvec, eigval, u = np.linalg.svd(CovM)

            # Get Semimajor/minor axes of the ellipse
            semimaj = np.sqrt(eigval[0])*2.
            semimin = np.sqrt(eigval[1])*2.

            # Rotation angle of the ellipse
            theta = np.arctan(eigvec[0][1]/eigvec[0][0])

            # Create ellipses
            ell = mpl.patches.Ellipse(xy=[...], width=1.52*semimaj, height=1.52*semimin, angle = theta*180/np.pi, facecolor = 'dodgerblue', edgecolor = 'royalblue', label = '68% confidence')
            ell2 = mpl.patches.Ellipse(xy=[...], width=2.48*semimaj, height=2.48*semimin, angle = theta*180/np.pi, facecolor = 'skyblue', edgecolor = 'royalblue', label = '95% confidence')

            ax.add_patch(ell2)
            ax.add_patch(ell)

            ax.set_ylim(...)
            ax.set_xlim(...)
            ax.set_xticks(...)
            ax.set_yticks(...)


        if j != 0:
            ax.set_yticklabels([])
        if i != 5:
            ax.set_xticklabels([])
        if j == 0 and i !=0:
            ax.set_ylabel(p_tex[i], fontsize=10)
            ax.set_yticklabels(...)
            [tl.set_rotation(26) for tl in ax.get_yticklabels()]
        if i == 5:
            ax.set_xlabel(p_tex[j], fontsize=10)
            ax.set_xticklabels(...)
            [tl.set_rotation(26) for tl in ax.get_xticklabels()]

In [ ]:
grader.check("q1.2")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Part 3. Markov Chain Monte Carlo



In many cases, the likelihood or the posterior distribution over parameter space is highly non-Gaussian.
When the Gaussian approximation is no longer sufficient for parameter distribution estimation and sampling, we can instead use Markov Chain Monte Carlo methods to do sampling to get a better estimate for the distribution of parameters.



Here, we will use the `pocomc` package as an example case for parameter sampling estimation. Following the example, you are expected to conduct the same type of sampling with your own Metropolis-Hastings algorithm, and get the same resulting plot.

So we define the priors as a list to the `pc.Prior` function, each item in the list should be a uniform random variable given by `uniform(lower, upper-lower)`.


In [ ]:
import pocomc as pc

In [ ]:
# Define prior

prior = pc.Prior([
    uniform(9,14-9), # logMmin in [9,14]
    uniform(0.01,1.5-0.01), # sigma_logM in [0.01,1.5]
    uniform(9,14-9), # logM0 in [9,14]
    uniform(10.7,15.0-10.7), # logM1 in [10.7,15.0]
    uniform(0,2-0), # alpha in [0,2]
    uniform(-1,+1-(-1)), # mean_occupation_centrals_assembias_param1 in [-1,+1]
    uniform(-1,+1-(-1)), # mean_occupation_satellites_assembias_param1 in [-1,+1]
])

In [ ]:
# Run pocoMC sampler (note that this should take ~ 5 minutes)

sampler = pc.Sampler(prior=prior, likelihood=log_likelihood)

sampler.run()

samples, weights, logl, logp = sampler.posterior()

logz, logz_err = sampler.evidence()

print('logZ = ', np.round(logz,4), '+-', np.round(logz_err,4))

In [ ]:
corner.corner(samples, weights=weights, color='C0', smooth=1.0,
              labels=['logMmin', 'sigma_logM', 'logM0', 'logM1', 'alpha', 'A_c', 'A_s'],
              range=[(9,14),(0.01,1.5),(9,14),(10.7,15.0),(0,2),(-1,+1),(-1,+1)]);

In [ ]:
for i in range(len(labels)):
    print("%s = %.5f +/- %.5f" %(..., ..., ...)))

In [ ]:
for i in range(len(labels)):
    print("%s : MLE values %.2f sigma away from MCMC mean" %(..., ...))

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

The results from this corner plot should look similar to your results from Part 2 using MLE. (If they don't, double check your log-likelihood function or your MLE code). We see however that the results are not quite Gaussian, especially given our priors. Our goal then is to reproduce this work using a simpler Markov Chain Monte Carlo algorithm: Metropolis Hastings.

**Write your own MCMC algorithm using Metropolis Hastings, and sample from the posterior distribution and make a corner plot like the one above. Report the mean and standard deviations of your sampled parameters, and plot the convergences of your chains as a function of timestep. Lastly, discuss the differences your simple MCMC alrogithm have with the pocomc plot above, the MLE plot from Part 2 as well as Figure 6 from https://arxiv.org/pdf/1606.07817. Remember to leave comments in your code and explain your steps.**

Hints:


1.   Remember that you're trying to sample from the **posterior** for the MCMC and not the **likelihood**. Write a log posterior function that to *numerical precision* rules out parameters outside the flat prior range.
2.   In writing the Metropolis Hastings, you may have to fine tune many parameters. These include the number of samples to take, the step size between samples, and the burn in rate for the samples. I suggest you write the code to make it easy to change these samples.



In [ ]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Problem 2: Predator - Prey Model


**1) Motivation**

Another application of a MC simulation is population dynamics. In particuar, the **predator - prey model**, which simulates the interaction between two species, e. g. wolfes and lambs was one of the early models that could explain periodic pattern in population sizes over time.<br> 
The goal of this exercise is to apply the MC simulation using the Gillespie algorithm to the predator - prey model and thereby gain more understanding of these methods.

**2) Preparation**

In its simplest version the predator - prey model consists of only three equations:<br>
<br>
- lambs $L$ double themselves with a rate $k_1$<br>
  $L \xrightarrow{k_1} 2\,L$<br>
- if a wolf $W$ meets a lamb $L$, it kills it and turns it into a new wolf with a rate $k_2$, such that<br>
 $L + W \xrightarrow{k_2} 2\,W$<br>
- Wolfs can starve (whereas lambs don't, they can always eat grass). If they do not meet a lamb, they will die with the rate $k_3$<br>
 $W \xrightarrow{k_3} \Phi$<br>

**3) Exercise**

- a) Write a Python script using *def* that simulates the predator - prey model from above via a MC simulation using the Gillespie algorithm. Start with the following values: $L(t = 0) = W(t = 0) = 1000$, $k_1 = 10$, $k_2 = 0.01$ and $k_3 = 10$. Experiment with sligthly different values.
- b) Plot $L(t)$ and $W(t)$, but also plot $L(W)$ vs $W(t)$. What do you observe?<br>
- c) In reality a wolf does not immediately turn a lamb into another wolf, but rather uses the energy for maintaining its metabolism. This would add another equation like $W + L \xrightarrow{k_4} W$ to the model. Also lambs can die by natural causes via $L \xrightarrow{k_5} \Phi$. Discuss (**no simulation is required!**) that adding these equations does not change the model at all. Apply what you know about rate equations.<br>

In [ ]:
def PredPrey(Lambs: int = 1000, Wolfes: int = 1000,
             k1: float = 10, k2: float = 0.01, k3: float = 10,
             Niter: int = 1000000):

    ...

    # Plotting code below: The function should plot the result, but not necessarily return anything

    # First, plot L(t) and W(t):
    fig = plt.figure(figsize=(8,6))
    ...

    plt.xlabel('time')
    plt.show()
    

    # Next, plot L(W):
    fig = plt.figure(figsize=(8,6))
    ...
    plt.xlabel('Wolfes')
    plt.ylabel('Lambs')    
    plt.show()

In [ ]:
PredPrey(Niter = 10000)

In [ ]:
PredPrey(Niter = 100000)

In [ ]:
PredPrey(Niter = 1000000)

<!-- END QUESTION -->



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit only the PDF to Gradescope! Do not submit the zip or ipynb files.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()